In [1]:
import salobj
import SALPY_LinearStage
import asyncio
import time

/home/saluser/lsst_repos/salobj/python/salobj/test_utils.py:37: UserWarning: Could not import SALPY_Test; TestComponent will not work
  warnings.warn("Could not import SALPY_Test; TestComponent will not work")


In [2]:
import logging
logging.getLogger().setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG,
                format='[%(asctime)s] [%(name)-12s:%(lineno)-4d] [%(levelname)-8s]: %(message)s',
                datefmt='%m-%d %H:%M:%S')

In [ ]:
class LinearStage:
    def __init__(self):
        self.linear_stage_1 = salobj.Remote(SALPY_LinearStage, f"LinearStage:1")
#         self.linear_stage_2 = salobj.Remote(SALPY_LinearStage, f"LinearStage:2")
    
    async def startup(self):

        logging.debug('Starting up Linear Stage...')
        
        # Tasks to get the summary state
        summary_state_1 = self.linear_stage_1.evt_summaryState.next(flush=True,
                                                                    timeout=30.)
        
        # Enter Control
        enterc_topic_1 = self.linear_stage_1.cmd_enterControl.DataType()
        task_enterc_1 = self.linear_stage_1.cmd_enterControl.start(enterc_topic_1)
        
        logging.debug('Entering control linear stages...')
        group_enterc = await asyncio.gather(summary_state_1, 
                                            task_enterc_1)
        logging.debug('Linear stages in standby')
        logging.debug('Linear stage 1 ack: %i' , group_enterc[1].ack)
        logging.debug('Linear stage 1 summaryState: %i', group_enterc[0].summaryState)

        # Starting
        summary_state_1 = self.linear_stage_1.evt_summaryState.next(flush=False,
                                                                    timeout=30.)

        start_topic_1 = self.linear_stage_1.cmd_start.DataType()
        task_start_1 = self.linear_stage_1.cmd_start.start(start_topic_1)
        
        logging.debug('Starting linear stages...')
        group_start = await asyncio.gather(summary_state_1, 
                                           task_start_1)

        logging.debug('Linear stages started')        
        logging.debug('Linear stage 1 ack: %i' , group_start[1].ack)
        logging.debug('Linear stage 1 summaryState: %i', group_start[0].summaryState)

        # Enabling
        summary_state_1 = self.linear_stage_1.evt_summaryState.next(flush=False,
                                                                    timeout=30.)
        position = self.linear_stage_1.tel_position.next(flush=False,
                                                         timeout=30.)
        enable_topic_1 = self.linear_stage_1.cmd_enable.DataType()
        task_enable_1 = self.linear_stage_1.cmd_enable.start(enable_topic_1)
        
        logging.debug('Enabling linear stages...')
        group_enable = await asyncio.gather(summary_state_1,
                                            task_enable_1, 
                                            position)
        logging.debug('Linear stages enabled')        
        logging.debug('Linear stage 1 ack: %i' , group_start[1].ack)
        logging.debug('Linear stage 1 summaryState: %i', group_start[0].summaryState)
        logging.debug('Linear stage 1 Position: %.3f', group_start[2].position)
        

In [4]:
loop = asyncio.new_event_loop()

In [4]:
seq = LinearStage()

In [5]:
print('Start')
task = asyncio.ensure_future(seq.startup())
print('Done')

Start
Done


[10-02 14:46:33] [root        :8   ] [DEBUG   ]: Starting up Linear Stage...
[10-02 14:46:33] [root        :18  ] [DEBUG   ]: Entering control linear stages...
[10-02 14:47:00] [root        :21  ] [DEBUG   ]: Linear stages in standby
[10-02 14:47:00] [root        :22  ] [DEBUG   ]: Linear stage 1 ack: 303
[10-02 14:47:00] [root        :23  ] [DEBUG   ]: Linear stage 1 summaryState: 2
[10-02 14:47:00] [root        :32  ] [DEBUG   ]: Starting linear stages...
[10-02 14:47:01] [root        :36  ] [DEBUG   ]: Linear stages started
[10-02 14:47:01] [root        :37  ] [DEBUG   ]: Linear stage 1 ack: 303
[10-02 14:47:01] [root        :38  ] [DEBUG   ]: Linear stage 1 summaryState: 4
[10-02 14:47:01] [root        :46  ] [DEBUG   ]: Enabling linear stages...
[10-02 14:47:01] [root        :49  ] [DEBUG   ]: Linear stages enabled
[10-02 14:47:01] [root        :50  ] [DEBUG   ]: Linear stage 1 ack: 303
[10-02 14:47:01] [root        :51  ] [DEBUG   ]: Linear stage 1 summaryState: 2
[10-02 14:47:01

In [6]:
task

<Task finished coro=<LinearStage.startup() done, defined at <ipython-input-3-c7c853e7f2ee>:6> exception=AttributeError("'SALPY_LinearStage.LinearStage_logevent_getPosition' object has no attribute 'position'",)>

In [7]:
position = asyncio.ensure_future(seq.linear_stage_1.tel_position.next())

In [8]:
p = asyncio.ensure_future(seq.linear_stage_1.evt_getPosition.next(flush=False))

In [9]:
p

<Task pending coro=<RemoteTelemetry._wait_next() running at /home/saluser/lsst_repos/salobj/python/salobj/remote_telemetry.py:219> wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x7f64ad66e738>()]>>

In [10]:
topic = seq.linear_stage_1.cmd_getHome.DataType()
home = asyncio.ensure_future(seq.linear_stage_1.cmd_getHome.start(topic))

In [ ]:
home.result().ack

In [12]:
position.result()

In [ ]:
a.priority

In [ ]:
topic = seq.linear_stage_1.cmd_moveRelative.DataType()
topic.distance = 30
move_task = asyncio.ensure_future(seq.linear_stage_1.cmd_moveRelative.start(topic))

In [ ]:
move_task

In [ ]:
position = asyncio.ensure_future(seq.linear_stage_1.tel_position.next())

In [ ]:
position.result().position